In [19]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import numpy as np

In [20]:
df = pd.read_csv('..\Data\Processed\processed_manga.csv')

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\Siddh\AppData\Local\Temp\ipykernel_8240\251128808.py:1: SyntaxWarning: invalid escape sequence '\D'
  df = pd.read_csv('..\Data\Processed\processed_manga.csv')


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 968 entries, 0 to 967
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         968 non-null    int64  
 1   title      968 non-null    object 
 2   synopsis   968 non-null    object 
 3   genres     968 non-null    object 
 4   score      968 non-null    float64
 5   image_url  968 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 45.5+ KB


In [22]:
df["content"] = (df["synopsis"]+ " ")*3 + " " + (df["genres"]+ " ")*2

In [23]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['content'])
print(tfidf_matrix.shape)


(968, 11929)


In [24]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print("Cosine similarity matrix shape:", cosine_sim.shape)


Cosine similarity matrix shape: (968, 968)


In [25]:
def get_recommendations(title, top_n=5):
    # Ensure title exists
    if title not in df['title'].values:
        raise ValueError(f"'{title}' not found in dataset.")
    
    # Get index of given title
    idx = df[df['title'] == title].index[0]
    
    # Pair index with similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort by similarity score (descending), skip the first one (itself)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    
    # Get the indices of recommended manga
    manga_indices = [i[0] for i in sim_scores]
    
    # Return a DataFrame of recommendations
    return df.iloc[manga_indices][['title', 'genres','synopsis', 'score','image_url']] if 'score' in df.columns else df.iloc[manga_indices][['title', 'genres']]


In [26]:
get_recommendations('Naruto')

,title,genres,synopsis,score,image_url
815,Tsuki no Shippo,Adventure Comedy Fantasy Romance,Usagi is the granddaughter of the leader of a ...,7.95,https://cdn.myanimelist.net/images/manga/3/160...
895,Shinobu Kokoro wa,Boys Love Erotica,The Ninja: A stealth fighter expertly trained ...,6.73,https://cdn.myanimelist.net/images/manga/2/357...
63,Ninin ga Shinobuden,Comedy,Shinobu is the only girl that goes to the secr...,6.82,https://cdn.myanimelist.net/images/manga/3/143...
498,Fuun Sanshimai Lin³,Adventure Fantasy,"Separated at birth, three Shaolin Princesses t...",6.43,https://cdn.myanimelist.net/images/manga/1/180...
395,Kagetora,Action Comedy Romance,The young ninja Kagetora has been given a grea...,7.47,https://cdn.myanimelist.net/images/manga/5/199...


In [27]:


# Save
with open("../model/tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(tfidf, f)

with open("../model/cosine_similarity.pkl", "wb") as f:
    pickle.dump(cosine_sim, f)

df.to_csv("../model/final_manga.csv", index=False)


FileNotFoundError: [Errno 2] No such file or directory: '../model/tfidf_vectorizer.pkl'

In [ ]:
ratings_df = pd.DataFrame({
    'user_id': [1, 1, 2, 2, 3],
    'manga_id': [1, 2, 2, 3, 1],
    'rating': [5, 4, 3, 4, 5]
})



In [ ]:
user_item_matrix = ratings_df.pivot_table(
    index='user_id',
    columns='manga_id',
    values='rating'
).fillna(0)


In [ ]:
print(user_item_matrix)

manga_id    1    2    3
user_id                
1         5.0  4.0  0.0
2         0.0  3.0  4.0
3         5.0  0.0  0.0


In [ ]:
import numpy as np
from scipy.sparse.linalg import svds

matrix = user_item_matrix.values

# Compute SVD
U, sigma, Vt = svds(matrix, k=2)
sigma = np.diag(sigma)

# Predicted ratings
predicted_ratings = np.dot(np.dot(U, sigma), Vt)
pred_df = pd.DataFrame(predicted_ratings, columns=user_item_matrix.columns, index=user_item_matrix.index)


In [ ]:
def get_cf_recommendations(user_id, top_n=5):
    user_preds = pred_df.loc[user_id].sort_values(ascending=False)
    rated = ratings_df[ratings_df.user_id == user_id]['manga_id'].tolist()
    recommended = user_preds.drop(rated).head(top_n)
    return df[df['id'].isin(recommended.index)][['id', 'title']]


In [ ]:
print(get_cf_recommendations(1))


   id              title
2   3  20th Century Boys


In [ ]:
import joblib

# Save CBF similarity matrix
joblib.dump(cosine_sim, "../models/cbf_cosine_matrix.pkl")
joblib.dump(tfidf, "../models/tfidf_vectorizer.pkl")

# Save CF SVD components
joblib.dump(U, "../models/cf_U.pkl")
joblib.dump(sigma, "../models/cf_sigma.pkl")
joblib.dump(Vt, "../models/cf_Vt.pkl")


['../models/cf_Vt.pkl']